In [1]:
# ignore waring 
import warnings
warnings.filterwarnings("ignore")

# Import packages needed
import time
import numpy as np
from math import pi
import paddle
from paddle_quantum.ansatz import Circuit
from paddle_quantum.linalg import dagger
from paddle_quantum.loss import ExpecVal
from paddle_quantum.state import zero_state

# Drawing tools
from matplotlib import pyplot as plt 

In [2]:
def rand_circuit(target, num_qubits, theta=None):
    # Initialize the quantum circuit
    cir = Circuit(num_qubits)
    
    # Fixed-angle Ry rotation gates 
    cir.ry(param=pi / 4)

    # ============== First layer ==============
    # Fixed-angle Ry rotation gates 
    for i in range(num_qubits):
        if target[i] == 0:
            cir.rz(i, param=theta[i] if theta is not None else theta)
        elif target[i] == 1:
            cir.ry(i, param=theta[i] if theta is not None else theta)
        else:
            cir.rx(i, param=theta[i] if theta is not None else theta)
            
    # ============== Second layer ==============
    # Build adjacent CZ gates
    for i in range(num_qubits - 1):
        cir.cz([i, i + 1])
        
    return cir

In [4]:
# Hyper parameter settings
# np.random.seed(42)   # Fixed Numpy random seed
N = 2                # Set the number of qubits
samples = 300        # Set the number of sampled random network structures
THETA_SIZE = N       # Set the size of the parameter theta
ITR = 1              # Set the number of iterations
LR = 0.2             # Set the learning rate
SEED = 1             # Fixed the randomly initialized seed in the optimizer

# Initialize the register for the gradient value
grad_info = []

# paddle.seed(SEED)
class manual_gradient(paddle.nn.Layer):
    
    # Initialize a list of learnable parameters and fill the initial value with a uniform distribution of [0, 2*pi]
    def __init__(self, shape, param_attr=paddle.nn.initializer.Uniform(low=0.0, high=2*pi), dtype='float32'):
        super(manual_gradient, self).__init__()
        
        # Convert Numpy array to Tensor in PaddlePaddle
        self.H = zero_state(N).data
        
    # Define loss function and forward propagation mechanism  
    def forward(self):
        
        # Initialize three theta parameter lists
        theta_np = np.random.uniform(low=0., high=2*pi, size=(THETA_SIZE))
        theta_plus_np = np.copy(theta_np) 
        theta_minus_np = np.copy(theta_np) 
        
        # Modified to calculate analytical gradient
        theta_plus_np[0] += np.pi/2
        theta_minus_np[0] -= np.pi/2
        
        # Convert Numpy array to Tensor in PaddlePaddle
        theta_plus = paddle.to_tensor(theta_plus_np)
        theta_minus = paddle.to_tensor(theta_minus_np)
        
        # Generate random targets, randomly select circuit gates in rand_circuit
        target = np.random.choice(3, N)      
        
        U_plus = rand_circuit(target, N, theta_plus).unitary_matrix()
        U_minus = rand_circuit(target, N, theta_minus).unitary_matrix()

        # Calculate the analytical gradient
        grad = paddle.real((dagger(U_plus) @ self.H @ U_plus)[0][0] - (dagger(U_minus) @ self.H @ U_minus)[0][0])/2  

        return grad

# Define the main block
def main():

    # Set the dimension of QNN
    sampling = manual_gradient(shape=[THETA_SIZE])
        
    # Sampling to obtain gradient information
    grad = sampling().numpy()
        
    return grad

# Record running time
time_start = time.time()

# Start sampling
for i in range(samples):
    if __name__ == '__main__':
        grad = main()
        grad_info.append(grad)

time_span = time.time() - time_start

print('The main program segment has run in total ', time_span, ' seconds')
print("Use ", samples, " samples to get the mean value of the gradient of the random network's first parameter, and we have：", np.mean(grad_info))
print("Use ", samples, "samples to get the variance of the gradient of the random network's first parameter, and we have：", np.var(grad_info))

I0217 13:58:03.811827 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.811858 3900496704 kernel_dispatch.h:120] promote kernel DataType:complex64
I0217 13:58:03.813606 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.813777 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.813990 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814471 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814525 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814586 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814672 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814915 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814952 3900496704 kernel_dispatch.h:102] Get BackendSet from tensor
I0217 13:58:03.814975 3900496704 kernel_dispatch.h:102] Get BackendSet from tens

ValueError: (InvalidArgument) The axis is expected to be in range of [0, 0), but got 0
  [Hint: Expected axis >= -rank && axis < rank == true, but received axis >= -rank && axis < rank:0 != true:1.] (at /Users/paddle/xly/workspace/293efbd7-945c-47ab-96a0-e0093f12eab2/Paddle/paddle/phi/infermeta/multiary.cc:1035)
